<img src='https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/banner_ikomia.png?raw=true'>

# How to train YOLOR using the Ikomia API

Object detection is an important task in Computer Vision. Lots of companies use this technology:


*   **Smart city** : crowd analysis or infrastructure inspection
*   **Retail** : inventory tracking or customer tracking
*   **Autonomous vehicles** : vehicle or pedestrian detection
*   **Security** : facial recognition or potential threats detection
*   **Manufacturing** : quality control or defect detection
*   **Healthcare** : many disease recognition


### Overview of the YOLOR model

| Release date | May 2021        |
|--------------|------------------|
| Model type   | Object detection |
| Architecture | YOLO, CNN        |
| Framework    | PyTorch          |
| Github stars | 1.9k             |



- Paper : [You Only Learn One Representation: Unified Network for Multiple Tasks](https://arxiv.org/abs/2105.04206)

- Code : [GitHub repository](https://github.com/WongKinYiu/yolor)

If you like this tutorial, you can support our project here [Ikomia API GitHub](https://github.com/Ikomia-dev/IkomiaApi).

ENJOY 🥰 !!

![YOLOR](https://github.com/WongKinYiu/yolor/raw/main/inference/output/horses.jpg)

## Setup

Please use a GPU for this tutorial.

In the menu, select "Runtime" then "Change runtime type", choose GPU in "Hardware accelerator".

Check your GPU with the following command:

In [ ]:
!nvidia-smi

First of all, you need to install Ikomia API pip package.

In [ ]:
!pip install ikomia



---


**-Google Colab ONLY- Restart runtime**

Click on the "RESTART RUNTIME" button at the end the previous window.





---

## How to train YOLOR on a custom COCO dataset

Download your dataset from your preferred tool. In this example, we use a construction safety dataset from **Roboflow** which is a great annotation platform used by many developers and companies. The dataset is exported in COCO format.

In [ ]:
import requests
import zipfile
import os


# Download the file
url = "https://universe.roboflow.com/ds/zC6XKnaUd7?key=bIco5taeND"
response = requests.get(url, stream=True)
with open("roboflow.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

# Unzip the file
with zipfile.ZipFile("roboflow.zip", 'r') as zip_ref:
    zip_ref.extractall()

# Remove the zip file
os.remove("roboflow.zip")

In order to train YOLOR on your custom dataset, please create a new workflow from scratch.

Then you need 2 components:

1.   A COCO dataset loader which loads dataset in YOLO format and convert it to an Ikomia format
2.   The YOLOR training algorithm which loads dataset in Ikomia format

Add these 2 previous algorithms to your workflow and then it will automagically download all algorithms from Ikomia Hub and install all the Python dependencies (the 1st time, it can take a while, be patient ! ).

Now, it's time to train your model !

In [ ]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils import ik
import os

#----------------------------- Step 1 -----------------------------------#
# Create a workflow which will take your dataset as input and
# train a YOLOR model on it
#------------------------------------------------------------------------#
wf = Workflow()

#----------------------------- Step 2 -----------------------------------#
# First you need to convert COCO format to IKOMIA format.
# Add an Ikomia dataset converter to your workflow.
#------------------------------------------------------------------------#

dataset = wf.add_task(name='dataset_coco')
dataset.set_parameters({
    'json_file': os.getcwd()+"/train/_annotations.coco.json",
    'image_folder': os.getcwd()+"/train",
    'task': 'detection',
}) 

#----------------------------- Step 3 -----------------------------------#
# Then, you want to train a YOLOR model.
# Add YOLOR training algorithm to your workflow
#------------------------------------------------------------------------#
train = wf.add_task(name='train_yolor', auto_connect=True)

train.set_parameters({
    'model_name': 'yolor_p6',
    'batch_size': '8', # <-- Reduce the batch size if you encounter some "Cuda out of memory"
    'epochs': '40',
    'train_imgsz': '512',
    'test_imgsz': '512',
    'dataset_split_ratio': '80',
    'eval_period': '5',
    'output_folder': os.getcwd(),
})

#----------------------------- Step 4 -----------------------------------#
# Execute your workflow.
# It automatically runs all your tasks sequentially.
#------------------------------------------------------------------------#
wf.run()

## How to execute YOLOR on images

Once the training is finished, you may want to experiment the fresh model on new test images. Just use the following code to create a YOLO v7 inference workflow.
Then run and test !

In [ ]:
from ikomia.dataprocess.workflow import Workflow

# Init your workflow
wf = Workflow()

# Add algorithm
yolor = wf.add_task(name="infer_yolor", auto_connect=True)

yolor.set_parameters({
    'model_weight_file': os.getcwd()+"/[timestamp]/weights/best.pt", # <-- replace "[timestamp]" by the correct folder name (/content/ for Google Colab)
    'config_file': os.getcwd()+"/[timestamp]/yolor_p6.cfg",  
})

# Run on your image  
wf.run_on(url="https://www.copelandintl.com/site/wp-content/uploads/2020/06/shutterstock_519019975.jpg")


## Display your results

In [ ]:
from ikomia.utils.displayIO import display
from PIL import ImageShow
ImageShow.register(ImageShow.IPythonViewer(), 0)

# Get YOLOR image result
img_bbox = yolor.get_image_with_graphics()

# Display in Colab
display(img_bbox)